## Analytical Hydrogen Orbital Plotting/Visualization

Prepared by Brenden Pelkie

Updated April 22, 2021

In this tutorial, the first few analytical solutions to the time independent Schrödinger equation (also known as orbitals or wave functions) will be presented and plotted. The derivation of the complete Hydrogen wave function will not be shown and can be found in most introductory quantum chemistry textbooks. 

Here, we will focus on the cases of zero magnetic quantum number to avoid complexities with complex plotting. Plots of the radial component of the wave function will be presented, as well as contour plots, 3D surface plots, and 3D isosurface plots of the complete wave function. 

These solutions are based on "Quantum Chemistry, 2nd. Ed" by Donald McQuarrie, and on the online lecture slides from PJ Grandinetti. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits import mplot3d
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import cm
import scipy.special as sp
import math
import plotly.graph_objects as go

font = {'weight' : 'bold',
        'size'   : 15}

matplotlib.rc('font', **font)



### Analytical eigenstates:

The eigenstates of the Hydrogen atom can be written as the product of two components: one radial and one angular. These components are specified by three quantum numbers: $n$, the principle quantum number, $l$, the angular momentum quantum number, and $m_l$, the magnetic quantum number. The angular component, denoted as $Y_l^{m_l}(\theta, \phi)$, is a spherical harmonic and depends on the angular quantum number $l$ and the magnetic quantum number $m_l$, as well as the angular spatial components $\theta$ and $\phi$. The radial component of the wave function is

$$
R_{n,l}=\left( \frac{2Z}{na_0} \right)^{l+3/2}\sqrt{\frac{(n-l-1)!}{2n(n+l)!}}r^l e^{-Zr/na_0} L_{n-l-1}^{2l+1} \left(\frac{2Z}{na_0}r\right)$$

Here, $a_0$ is the Bohr radius (1 in atomic units), Z is the charge of the nucleus (also 1 for Hydrogen), and L is a Laguerre polynomial.

### Defining Wave Equations in Python:
Hereafter we define functions for the angular, radial, and complete wave functions as well as functions to plot these. 



In [ ]:
# Define wave function for orbital:

def radial(n,l,r):
    '''
    Computes the radial component of the wave equation for the Hydrogen Atom
    
    Parameters
    ----------
        r - radial coordinates. Accepts 1-d vector or 3D meshgrid coordinates.
        n - priciple quantum number
        l - angular momentum quantum number. Must be less than n.
           
    Returns:
    ----------
        R - radial component of the wave function
            
    '''
    #### Check input: from selection rules, n>l ####
    
    if l>(n-1):
        raise Exception("l must be less than or equal to n-1. Please review available angular momentum quantum numbers")
        
        
    #### Define Constants ####
    
    a0=1 # bohr radius in AU
    Z=1  # nuclear charge
    
    
    #### Radial Component of Wave function from PJ Grandinetti slides ####
    
    R=-(((math.factorial(n-l-1))/(2*n*(math.factorial(n+l))))**(1/2))*((2/(n*a0))**(l+3/2))*(r**l)*(np.exp(-r/(n*a0)))*sp.eval_genlaguerre((n-l-1),(2*l+1),(2*r/(n*a0)))
    
    
    #### The following code is an alternative that uses the analytically simplified radial wave functions, as found in table 7.2 of McQuarrie. These can be used if the above general radial wave function is not working as expected.####
    """
    if n>3:
        raise Exception("Max n is 3")

    
    if n==1:
        R=2*(Z/a0)**(3/2)*np.exp(-rho)
    if n==2:
        if l==0:
            R=(Z/(2*a0))**(3/2)*(2-rho)*np.exp(-rho/2)
        if l==1:
            R=(1/np.sqrt(3))*(Z/(2*a0))**(3/2)*rho*np.exp(-rho/2)
    if n==3:
        if l==0:
            R=(2/27)*(Z/(3*a0))**(3/2)*(27-18*rho+2*rho**2)*np.exp(-rho/3)
        if l==1:
               R=(1/27)*(2*Z/(3*a0))**(3/2)*rho*(6-rho)*np.exp(-rho/3)
        if l==2:
               R=(4/(27*np.sqrt(10)))*(Z/(3*a0))**(3/2)*rho**2*np.exp(-rho/3)
    """
    
    return(R)

def angular(l,theta,phi):
    """
    Computes angular component of Hydrogen atomic wave function.
    
    Parameters:
    ----------
        n - principle quantum number
        l - angular momentum quantum number
        ml - magnetic quantum number
        theta - theta space as 1D vector or meshgrid
        phi - phi space as 1D Vector or meshgrid
    
    Returns:
    ----------
        Y - spherical harmonic corresponding to input quantum numbers. This is the angular component of the wave function.   
            
    """
    
    Y=np.real(sp.sph_harm(0,l,0,theta))
    
    return(Y)

def fullWF(n,l,dist=25):
    """
    Computes complete wave function for atomic hydrogen for desired n and l, and ml=0. 
    Computes properties on a 50x50x50 au 3D cartesian grid, centered on the nucleus.
    
    Inputs: n - principle quantum number
            l - angular momentum quantum number
            ml - magnetic quantum number
            
    Returns: Psi - complete H atomic wave function, slice taken at phi=0.001.
             r
             theta
             phi
    """
    
    #### Define Space ####
    
    # set spatial extent with dist: Space can be expanded/shrunk here
    
    
    
    x0=np.linspace(-dist,dist,100)
    y0=np.linspace(-dist,dist,100)
    z0=np.linspace(-dist,dist,100)

    # using meshgrid allows the 3D calculations to work right
    x,y,z=np.meshgrid(x0,y0,z0)
    
    
    # cartesian coordinates are converted to spherical coordinates for use in calculations. Cartesian grid topology maintained.
    r,theta,phi=cartesian_to_spherical(x,y,z)
    
    
    #### Compute angular and radial parts of wave function ####
    
    R=radial(n,l,r)
    Y=angular(l,theta,phi)
    """
    ####The general equations don't seem to be working - try again with functions out of book
    
    #ml=0 only
    a0=1
    Z=1 #nuclear charge
    rho=Z*r/a0
    
    if n>3:
        raise Exception("Max n is 3")
    if l>(n-1):
        raise Exception("l must be less than or equal to n-1. Please review available angular momentum quantum numbers")
    
    
    if n==1:
        Psi=(1/np.sqrt(np.pi))*(Z/a0)**(3/2)*np.exp(-rho)
        
    if n==2:
        if l==0:
            Psi=(1/(4*np.sqrt(2*np.pi)))*(Z/a0)**(3/2)*(2-rho)*np.exp(-rho/2)
        if l==1:
            Psi=(1/(4*np.sqrt(2*np.pi)))*(Z/a0)**(3/2)*rho*np.exp(-rho/2)*np.cos(theta)
    if n==3:
        if l==0:
            Psi=(1/(81*np.sqrt(3*np.pi)))*(Z/(a0))**(3/2)*(27-18*rho+2*rho**2)*np.exp(-rho/3)
                 
        if l==1:
            Psi=(np.sqrt(2)/(81*np.sqrt(np.pi)))*(Z/a0)**(3/2)*rho**2*np.exp(-rho/3)*np.cos(theta)
                 
        
        if l==2:
            Psi=(1/(81*np.sqrt(np.pi*6)))*(Z/a0)**(3/2)*rho**2*np.exp(-rho/3)*(3*np.cos(theta)**2-1)
        """
    
    Psi=R*Y
    
    
    return(Psi,x,y,z)


#### Coordinate converting utilities ####

#def spherical_to_cartesian(r, theta, phi):
#    x=r*np.cos(phi)*np.sin(theta)
#    y=r*np.sin(phi)*np.sin(theta)
#    z=r*np.cos(theta)
#    
#    return(x,y,z)

def cartesian_to_spherical(x,y,z):
    """
    Converts a 3D grid of cartesian coordinates to spherical coordinates, maintains shape of cartesian grid (ie, only changes values at each point to correspons to r, theta, phi). This will return phi as the elevation coordinate, ranging from 0 to Pi, and theta as the azimuthal coordinate, ranging from zero to 2Pi.
    
    Parameters:
    ----------
        x - grid of x coordinates. Should be 3D meshgrid, but will work otherwise
        y - --""--  y --------------------------------""-------------------------
        z - --""--  z --------------------------------""-------------------------
        
    Returns:
    ----------
        r - grid of radial coordinates
        theta - grid of azimuthal coordinates, on [0,2Pi]
        phi - Elevation coordinates
    """
    r=np.sqrt(x**2+y**2+z**2)
    theta=np.arctan2(y,x)
    #phi=np.arctan(np.sqrt(x**2+y**2)/z)
    phi=np.arccos(z/(np.sqrt(x**2+y**2+z**2)))
    #this will make theta range from zero to 2*Pi
    theta[theta<0]+=2*np.pi
    return(r,theta,phi)



### Radial Wave Function Plots:

First, plot the radial wave function. This is done by calling the radial function from above with the appropriate quantum numbers. The normalized probability density is actually plotted, which is effectively $r^2R^2$ in au. This is plotted on a straightforward scatter plot.

In [ ]:
r=np.linspace(0,25,500)

#1s radial
rad_1s=radial(1,0,r)

#2s orbital
rad_2s=radial(2,0,r)

#2p orbital
rad_2p=radial(2,1,r)

#3s orbital
rad_3s=radial(3,0,r)

#3p orbital
rad_3p=radial(3,1,r)

#3d orbital
rad_3d=radial(3,2,r)

#plot wave function listed in book:


fig,ax=plt.subplots(figsize=(20,12))
ax.plot(r,rad_1s**2*r**2,label="1s")

ax.plot(r,rad_2s**2*r**2, label="2s")
ax.plot(r, rad_2p**2*r**2, label = '2p')
ax.plot(r, rad_3s**2*r**2, label='3s')
ax.plot(r, rad_3p**2*r**2, label='3p')
ax.plot(r, rad_3d**2*r**2, label='3d')
ax.set_title("Normalized Probability Density of Radial Wave Function")
ax.set_xlabel('Distance [Bohr Radii]')
ax.set_ylabel('Probability')
ax.legend()

### Complete Wave Function Contour Plots:

Next, we will create contour plots of the composite wave function. These are plots of a slice that is taken looking "down" on the orbitals, at a fixed z location that is as close to zero as we can get without introducing divide by zero errors.

In [ ]:

def contourplot(n,l):
    """
    Generates contour plot of orbital density near z=0 axis, uses compositeWF to generate Psi
    
    Parameters:
    ----------
        n - principle quantum number
        l - angular momentum quantum number
    
    """
    Psi, r, theta, phi = compositeWF(n,l)


    #contour plot of the orbital 
    #fig, ax = plt.subplots(figsize=(10,10),subplot_kw=dict(projection='polar'))
    fig,ax=plt.subplots(figsize=(12,10))
    CS=ax.contourf(theta[:,:,49], r[:,:,49], Psi[:,:,49]**2,100)
    ax.grid(False)
    ax.set_xticks([-25,0,25])
    ax.set_yticks([-25,0,25])
    ax.set_xlabel('X direction [Bohr Radii]')
    ax.set_ylabel('Y direction [Bohr Radii]')
    cbar=fig.colorbar(CS)
    ax.set_title(f"Contour Slice of Orbital, n={n}, l={l}")


In [ ]:
for n in [1,2,3]:
    if n==1:
        l=0
        contourplot(n,l)
    if n==2:
        l=0
        contourplot(n,l)
        l=1
        contourplot(n,l)
    if n==3:
        for l in [0,1,2]:
            contourplot(n,l)

### 3D surface plot:

This plot shows the same information as in the contour plots, but with a 3D surface plot. The contour lines are projected onto the z plane. These graphics help to better visualize the wave functions. To change the plotted orbital, change the values of n and l in the call to compositeWF on line 3.



In [ ]:
####Call compositeWF to get desired orbital:####

Psi,x,y,z=compositeWF(3,2,15)

####Create 3D surface plot####

fig = plt.figure(figsize=(10,10))

ax=fig.gca(projection='3d')

ax.set_xlim(-15,15)
ax.set_ylim(-15,15)
ax.set_xlabel('x [au]')
ax.set_ylabel('y [au]')
ax.set_zlabel('$\Psi^2$')

ax.zaxis.labelpad = 20
CP=ax.plot_surface(x[:,:,50],y[:,:,50],Psi[:,:,50]**2, alpha=0.5, cmap=cm.binary)

ax.view_init(elev=30,azim=35)
cbar=fig.colorbar(CP,fraction=.03, pad=0.04)
cbar.set_label('$\Psi^2$', rotation=90)
cset=ax.contour(x[:,:,49], y[:,:,49], Psi[:,:,49]**2,10, zdir='z', offset=0, cmap=cm.binary)


    


### 3D Isosurface Plot:

While the above surface plot is 3D, it is only showing 2D orbital information, as we took a slice through a constant z value. To show the values of the wave function in 3 dimensions, we can use an Isosurface plot. An isosurface plot draws a surface through space where the value of $\Psi$ is constant. It is similar to a contour plot, but in 3D. To generate isosurface plots, we will use the Plotly package. Here, we will feed the Plotly isosurface function the full X, Y, Z and Psi matrices in 3D, as well as minimum and maximum values of Psi where we would like isosurfaces to be drawn. Again, to change the visualized orbital, change the quantum numbers in the compositeWF call.


In [ ]:
Psi,x,y,z=compositeWF(3,2)

values=Psi**2

#plotly isosurface plotting

fig = go.Figure(data=go.Isosurface(
    x=x.flatten(),
    y=y.flatten(),
    z=z.flatten(),
    value=values.flatten(),
    isomin=0.0001,
    isomax=0.0007,
    surface_count=3,
    opacity=0.4,
    caps=dict(x_show=False, y_show=False, z_show=False),
    colorbar=dict(title="Psi^2")

    )
    )
fig.update_layout(scene=dict(
    xaxis_title="x [au]",
    yaxis_title="y [au]",
    zaxis_title='z [au]'))

fig.update_coloraxes(colorbar_title=dict(text='Psi^2', side='right'))
    



fig.show()